# Medicare Repricer - Visual Testing Notebook

This notebook provides interactive testing and visualization of the Medicare Claims Repricing system.

## Overview
The Medicare Repricer calculates Medicare allowed amounts based on:
- **RVU** (Relative Value Units) - Work, Practice Expense, Malpractice
- **GPCI** (Geographic Practice Cost Indices) - Location-based adjustments
- **Conversion Factor** - Multiplier to convert RVUs to dollar amounts
- **Modifiers** - Professional/technical components, bilateral procedures
- **Place of Service** - Facility vs non-facility settings

## Setup

In [ ]:
# Import required libraries
import sys
from pathlib import Path
import pandas as pd
import matplotlib.pyplot as plt
import seaborn as sns

# Set display options
pd.set_option('display.max_columns', None)
pd.set_option('display.width', None)
pd.set_option('display.max_colwidth', None)

# Style settings
sns.set_style('whitegrid')
plt.rcParams['figure.figsize'] = (12, 6)

# Import Medicare Repricer
from medicare_repricing import MedicareRepricer, Claim, ClaimLine

print("✓ Setup complete")

## Initialize Repricer

Load the Medicare fee schedule data (RVU and GPCI values) and initialize the repricer.

In [ ]:
# Initialize the repricer with actual 2025 Medicare data
data_dir = Path('data/medicare_2025')

if data_dir.exists():
    repricer = MedicareRepricer(data_directory=data_dir)
    print(f"✓ Loaded 2025 Medicare fee schedule from {data_dir}")
else:
    repricer = MedicareRepricer()
    print("✓ Using default sample fee schedule")

print(f"  Conversion Factor: ${repricer.fee_schedule.conversion_factor}")

## Test 1: Simple Office Visit

Reprice a basic office visit with a common procedure code.

In [ ]:
# Create a simple claim
claim = Claim(
    claim_id="DEMO001",
    lines=[
        ClaimLine(
            line_number=1,
            procedure_code="99213",  # Office visit, established patient
            modifier=None,
            place_of_service="11",  # Office
            locality="00",  # National average
            units=1
        )
    ]
)

# Reprice the claim
repriced = repricer.reprice_claim(claim)

# Display results
print(f"Claim ID: {repriced.claim_id}")
print(f"\nProcedure: {repriced.lines[0].procedure_code}")
print(f"Place of Service: {repriced.lines[0].place_of_service}")
print(f"Locality: {repriced.lines[0].locality}")
print("\n" + "="*60)
print("RVU Components:")
print(f"  Work RVU:              {repriced.lines[0].work_rvu:>8.2f}")
print(f"  Practice Expense RVU:  {repriced.lines[0].pe_rvu:>8.2f}")
print(f"  Malpractice RVU:       {repriced.lines[0].mp_rvu:>8.2f}")
print("\nGPCI Adjustments:")
print(f"  Work GPCI:             {repriced.lines[0].work_gpci:>8.3f}")
print(f"  PE GPCI:               {repriced.lines[0].pe_gpci:>8.3f}")
print(f"  MP GPCI:               {repriced.lines[0].mp_gpci:>8.3f}")
print("\nPricing:")
print(f"  Conversion Factor:     ${repriced.lines[0].conversion_factor:>8.4f}")
print(f"  Medicare Allowed:      ${repriced.lines[0].medicare_allowed:>8.2f}")
print("="*60)
print(f"\nTotal Allowed: ${repriced.total_allowed:.2f}")

## Test 2: Geographic Comparison

Compare the same procedure across different geographic locations.

In [ ]:
# Define localities to compare
localities = [
    ("00", "National Average"),
    ("01", "Manhattan, NY"),
    ("26", "Dallas, TX"),
    ("18", "Los Angeles, CA"),
]

results = []

for locality_code, locality_name in localities:
    claim = Claim(
        claim_id=f"GEO-{locality_code}",
        lines=[
            ClaimLine(
                line_number=1,
                procedure_code="99214",  # Office visit, detailed
                modifier=None,
                place_of_service="11",
                locality=locality_code,
                units=1
            )
        ]
    )
    
    repriced = repricer.reprice_claim(claim)
    line = repriced.lines[0]
    
    results.append({
        'Locality': locality_name,
        'Code': locality_code,
        'Work GPCI': line.work_gpci,
        'PE GPCI': line.pe_gpci,
        'MP GPCI': line.mp_gpci,
        'Allowed Amount': line.medicare_allowed
    })

# Create DataFrame
df_geo = pd.DataFrame(results)
print("\nGeographic Comparison for CPT 99214 (Office Visit, Detailed)\n")
print(df_geo.to_string(index=False))

# Visualize
fig, (ax1, ax2) = plt.subplots(1, 2, figsize=(14, 5))

# GPCI comparison
df_geo.set_index('Locality')[['Work GPCI', 'PE GPCI', 'MP GPCI']].plot(
    kind='bar', ax=ax1, rot=45
)
ax1.set_title('GPCI Values by Locality')
ax1.set_ylabel('GPCI Value')
ax1.legend(loc='best')
ax1.axhline(y=1.0, color='r', linestyle='--', alpha=0.3, label='National Average')

# Allowed amount comparison
colors = ['#1f77b4', '#ff7f0e', '#2ca02c', '#d62728']
ax2.bar(df_geo['Locality'], df_geo['Allowed Amount'], color=colors)
ax2.set_title('Medicare Allowed Amount by Locality')
ax2.set_ylabel('Allowed Amount ($)')
ax2.set_xticklabels(df_geo['Locality'], rotation=45, ha='right')

# Add value labels on bars
for i, v in enumerate(df_geo['Allowed Amount']):
    ax2.text(i, v + 1, f'${v:.2f}', ha='center', va='bottom')

plt.tight_layout()
plt.show()

## Test 3: Facility vs Non-Facility

Compare pricing between office (non-facility) and hospital outpatient (facility) settings.

In [ ]:
# Test multiple procedures in different settings
procedures = [
    ("99213", "Office Visit - Level 3"),
    ("99214", "Office Visit - Level 4"),
    ("99203", "New Patient - Level 3"),
]

results = []

for proc_code, proc_desc in procedures:
    # Office (non-facility)
    claim_office = Claim(
        claim_id=f"FAC-{proc_code}-OFFICE",
        lines=[
            ClaimLine(
                line_number=1,
                procedure_code=proc_code,
                modifier=None,
                place_of_service="11",  # Office
                locality="00",
                units=1
            )
        ]
    )
    
    # Hospital (facility)
    claim_hospital = Claim(
        claim_id=f"FAC-{proc_code}-HOSP",
        lines=[
            ClaimLine(
                line_number=1,
                procedure_code=proc_code,
                modifier=None,
                place_of_service="22",  # Hospital Outpatient
                locality="00",
                units=1
            )
        ]
    )
    
    repriced_office = repricer.reprice_claim(claim_office)
    repriced_hospital = repricer.reprice_claim(claim_hospital)
    
    office_line = repriced_office.lines[0]
    hospital_line = repriced_hospital.lines[0]
    
    results.append({
        'Procedure': proc_desc,
        'Code': proc_code,
        'Office PE RVU': office_line.pe_rvu,
        'Office Amount': office_line.medicare_allowed,
        'Hospital PE RVU': hospital_line.pe_rvu,
        'Hospital Amount': hospital_line.medicare_allowed,
        'Difference': office_line.medicare_allowed - hospital_line.medicare_allowed
    })

df_fac = pd.DataFrame(results)
print("\nFacility vs Non-Facility Comparison\n")
print(df_fac.to_string(index=False))

# Visualize
fig, ax = plt.subplots(figsize=(12, 6))

x = range(len(df_fac))
width = 0.35

ax.bar([i - width/2 for i in x], df_fac['Office Amount'], width, 
       label='Office (Non-Facility)', color='#2ca02c')
ax.bar([i + width/2 for i in x], df_fac['Hospital Amount'], width, 
       label='Hospital (Facility)', color='#d62728')

ax.set_xlabel('Procedure')
ax.set_ylabel('Medicare Allowed Amount ($)')
ax.set_title('Facility vs Non-Facility Medicare Allowed Amounts')
ax.set_xticks(x)
ax.set_xticklabels(df_fac['Procedure'], rotation=45, ha='right')
ax.legend()

# Add value labels
for i in x:
    ax.text(i - width/2, df_fac.iloc[i]['Office Amount'] + 1, 
            f"${df_fac.iloc[i]['Office Amount']:.2f}", 
            ha='center', va='bottom', fontsize=9)
    ax.text(i + width/2, df_fac.iloc[i]['Hospital Amount'] + 1, 
            f"${df_fac.iloc[i]['Hospital Amount']:.2f}", 
            ha='center', va='bottom', fontsize=9)

plt.tight_layout()
plt.show()

## Test 4: Modifiers

Test different modifiers (26: Professional Component, TC: Technical Component, 50: Bilateral).

In [ ]:
# Test modifiers on a radiology procedure
procedure_code = "71046"  # Chest X-ray
modifiers = [
    (None, "Global (Complete)"),
    ("26", "Professional Component"),
    ("TC", "Technical Component"),
]

results = []

for modifier, description in modifiers:
    claim = Claim(
        claim_id=f"MOD-{modifier or 'NONE'}",
        lines=[
            ClaimLine(
                line_number=1,
                procedure_code=procedure_code,
                modifier=modifier,
                place_of_service="22",  # Hospital
                locality="00",
                units=1
            )
        ]
    )
    
    repriced = repricer.reprice_claim(claim)
    line = repriced.lines[0]
    
    results.append({
        'Modifier': modifier or 'None',
        'Description': description,
        'Work RVU': line.work_rvu,
        'PE RVU': line.pe_rvu,
        'MP RVU': line.mp_rvu,
        'Allowed Amount': line.medicare_allowed
    })

df_mod = pd.DataFrame(results)
print(f"\nModifier Comparison for CPT {procedure_code} (Chest X-ray)\n")
print(df_mod.to_string(index=False))

# Visualize RVU components
fig, (ax1, ax2) = plt.subplots(1, 2, figsize=(14, 5))

# RVU breakdown
df_mod.set_index('Description')[['Work RVU', 'PE RVU', 'MP RVU']].plot(
    kind='bar', stacked=True, ax=ax1, rot=45
)
ax1.set_title('RVU Components by Modifier')
ax1.set_ylabel('RVU Value')
ax1.legend(title='RVU Type')

# Allowed amounts
colors = ['#1f77b4', '#ff7f0e', '#2ca02c']
ax2.bar(df_mod['Description'], df_mod['Allowed Amount'], color=colors)
ax2.set_title('Medicare Allowed Amount by Modifier')
ax2.set_ylabel('Allowed Amount ($)')
ax2.set_xticklabels(df_mod['Description'], rotation=45, ha='right')

# Add value labels
for i, v in enumerate(df_mod['Allowed Amount']):
    ax2.text(i, v + 1, f'${v:.2f}', ha='center', va='bottom')

plt.tight_layout()
plt.show()

# Verify that Global = Professional + Technical
global_amt = df_mod[df_mod['Modifier'] == 'None']['Allowed Amount'].values[0]
prof_amt = df_mod[df_mod['Modifier'] == '26']['Allowed Amount'].values[0]
tech_amt = df_mod[df_mod['Modifier'] == 'TC']['Allowed Amount'].values[0]

print(f"\nVerification: Global = Professional + Technical")
print(f"  Global: ${global_amt:.2f}")
print(f"  Prof + Tech: ${prof_amt:.2f} + ${tech_amt:.2f} = ${prof_amt + tech_amt:.2f}")
print(f"  Difference: ${abs(global_amt - (prof_amt + tech_amt)):.2f}")

## Test 5: Multiple Procedures (MPPR)

Test Multiple Procedure Payment Reduction (MPPR) logic.

In [ ]:
# Create a claim with multiple procedures
claim = Claim(
    claim_id="MPPR-TEST",
    lines=[
        ClaimLine(
            line_number=1,
            procedure_code="12002",  # Simple repair, higher RVU
            modifier=None,
            place_of_service="11",
            locality="00",
            units=1
        ),
        ClaimLine(
            line_number=2,
            procedure_code="12001",  # Simple repair, lower RVU
            modifier=None,
            place_of_service="11",
            locality="00",
            units=1
        ),
    ]
)

repriced = repricer.reprice_claim(claim)

# Display results
results = []
for line in repriced.lines:
    results.append({
        'Line': line.line_number,
        'Procedure': line.procedure_code,
        'Work RVU': line.work_rvu,
        'PE RVU': line.pe_rvu,
        'MP RVU': line.mp_rvu,
        'Allowed Amount': line.medicare_allowed,
        'Adjustment': line.adjustment_reason or 'None'
    })

df_mppr = pd.DataFrame(results)
print("\nMultiple Procedure Payment Reduction (MPPR) Test\n")
print(df_mppr.to_string(index=False))
print(f"\nTotal Allowed: ${repriced.total_allowed:.2f}")

# Display notes
print("\nClaim Notes:")
for note in repriced.notes:
    print(f"  • {note}")

# Visualize
fig, ax = plt.subplots(figsize=(10, 6))

x = range(len(df_mppr))
colors = ['#2ca02c', '#ff7f0e']

bars = ax.bar(x, df_mppr['Allowed Amount'], color=colors)

# Add labels
for i, (idx, row) in enumerate(df_mppr.iterrows()):
    ax.text(i, row['Allowed Amount'] + 2, f"${row['Allowed Amount']:.2f}",
            ha='center', va='bottom', fontweight='bold')
    if row['Adjustment'] != 'None':
        ax.text(i, row['Allowed Amount'] / 2, 'MPPR Applied',
                ha='center', va='center', rotation=90, color='white', fontweight='bold')

ax.set_xlabel('Procedure')
ax.set_ylabel('Medicare Allowed Amount ($)')
ax.set_title('Multiple Procedure Payment Reduction (MPPR) Example')
ax.set_xticks(x)
ax.set_xticklabels([f"{row['Procedure']}\nLine {row['Line']}" for _, row in df_mppr.iterrows()])

plt.tight_layout()
plt.show()

## Test 6: Units Multiplier

Test how units affect the total allowed amount.

In [ ]:
# Test different unit values
procedure_code = "99213"
units_to_test = [1, 2, 3, 5, 10]

results = []

for units in units_to_test:
    claim = Claim(
        claim_id=f"UNITS-{units}",
        lines=[
            ClaimLine(
                line_number=1,
                procedure_code=procedure_code,
                modifier=None,
                place_of_service="11",
                locality="00",
                units=units
            )
        ]
    )
    
    repriced = repricer.reprice_claim(claim)
    
    results.append({
        'Units': units,
        'Allowed Amount': repriced.total_allowed,
        'Per Unit': repriced.total_allowed / units
    })

df_units = pd.DataFrame(results)
print(f"\nUnits Multiplier Test for CPT {procedure_code}\n")
print(df_units.to_string(index=False))

# Visualize
fig, ax = plt.subplots(figsize=(10, 6))

ax.plot(df_units['Units'], df_units['Allowed Amount'], marker='o', 
        linewidth=2, markersize=8, color='#1f77b4')
ax.set_xlabel('Number of Units')
ax.set_ylabel('Total Medicare Allowed Amount ($)')
ax.set_title(f'Linear Relationship: Units vs Allowed Amount (CPT {procedure_code})')
ax.grid(True, alpha=0.3)

# Add value labels
for _, row in df_units.iterrows():
    ax.annotate(f"${row['Allowed Amount']:.2f}", 
                xy=(row['Units'], row['Allowed Amount']),
                xytext=(5, 5), textcoords='offset points',
                fontsize=9)

plt.tight_layout()
plt.show()

# Verify linearity
per_unit_avg = df_units['Per Unit'].mean()
per_unit_std = df_units['Per Unit'].std()
print(f"\nPer-Unit Amount: ${per_unit_avg:.2f} ± ${per_unit_std:.4f}")
print("(Should be constant, verifying linear relationship)")

## Test 7: Custom Claim

Create and test your own custom claim below.

In [ ]:
# Create your custom claim here
custom_claim = Claim(
    claim_id="CUSTOM001",
    lines=[
        ClaimLine(
            line_number=1,
            procedure_code="99213",  # Change this to your procedure code
            modifier=None,  # Options: None, "26", "TC", "50", etc.
            place_of_service="11",  # "11" = Office, "22" = Hospital
            locality="00",  # "00" = National, "01" = Manhattan, "26" = Dallas, etc.
            units=1
        ),
        # Add more lines if needed
    ]
)

# Reprice the custom claim
custom_repriced = repricer.reprice_claim(custom_claim)

# Display results
print(f"\nCustom Claim Results: {custom_repriced.claim_id}\n")
print("="*70)

for line in custom_repriced.lines:
    print(f"\nLine {line.line_number}: {line.procedure_code}" + 
          (f"-{line.modifier}" if line.modifier else ""))
    print(f"  Place of Service: {line.place_of_service}")
    print(f"  Locality: {line.locality}")
    print(f"  Units: {line.units}")
    print(f"\n  RVUs:")
    print(f"    Work:  {line.work_rvu:>6.2f}  (GPCI: {line.work_gpci:.3f})")
    print(f"    PE:    {line.pe_rvu:>6.2f}  (GPCI: {line.pe_gpci:.3f})")
    print(f"    MP:    {line.mp_rvu:>6.2f}  (GPCI: {line.mp_gpci:.3f})")
    print(f"\n  Medicare Allowed: ${line.medicare_allowed:.2f}")
    if line.adjustment_reason:
        print(f"  Adjustment: {line.adjustment_reason}")

print("\n" + "="*70)
print(f"Total Medicare Allowed: ${custom_repriced.total_allowed:.2f}")
print("="*70)

if custom_repriced.notes:
    print("\nNotes:")
    for note in custom_repriced.notes:
        print(f"  • {note}")

## Query Fee Schedule

Look up information about specific procedure codes and localities.

In [ ]:
# Query a procedure code
procedure_code = "99214"  # Change this to any procedure code

proc_info = repricer.get_procedure_info(procedure_code)

if proc_info:
    print(f"\nProcedure Information: {procedure_code}\n")
    print("="*60)
    print(f"Description: {proc_info['description']}")
    print(f"\nNon-Facility RVUs:")
    print(f"  Work: {proc_info['work_rvu_non_facility']:.2f}")
    print(f"  PE:   {proc_info['pe_rvu_non_facility']:.2f}")
    print(f"  MP:   {proc_info['mp_rvu_non_facility']:.2f}")
    print(f"\nFacility RVUs:")
    print(f"  Work: {proc_info['work_rvu_facility']:.2f}")
    print(f"  PE:   {proc_info['pe_rvu_facility']:.2f}")
    print(f"  MP:   {proc_info['mp_rvu_facility']:.2f}")
    print(f"\nMPPR Indicator: {proc_info['mppr_indicator']}")
    print(f"Conversion Factor: ${proc_info['conversion_factor']:.4f}")
    print("="*60)
else:
    print(f"Procedure code {procedure_code} not found in fee schedule.")

In [ ]:
# Query a locality
locality_code = "01"  # Change this to any locality code

loc_info = repricer.get_locality_info(locality_code)

if loc_info:
    print(f"\nLocality Information: {locality_code}\n")
    print("="*60)
    print(f"Name: {loc_info['locality_name']}")
    print(f"\nGPCI Values:")
    print(f"  Work GPCI: {loc_info['work_gpci']:.3f}")
    print(f"  PE GPCI:   {loc_info['pe_gpci']:.3f}")
    print(f"  MP GPCI:   {loc_info['mp_gpci']:.3f}")
    print("="*60)
else:
    print(f"Locality code {locality_code} not found in fee schedule.")

## Summary

This notebook demonstrated:
1. ✓ Simple office visit repricing
2. ✓ Geographic variations (GPCI adjustments)
3. ✓ Facility vs non-facility pricing
4. ✓ Modifier handling (26, TC, 50)
5. ✓ Multiple Procedure Payment Reduction (MPPR)
6. ✓ Units multiplier
7. ✓ Fee schedule queries

### Key Takeaways
- Medicare pricing is based on RVUs adjusted by geographic GPCI values
- Only **procedure code, modifiers, place of service, and locality** are needed for repricing
- Patient ID and diagnosis codes are NOT required for repricing calculations
- Facility settings generally pay less than non-facility due to lower Practice Expense RVUs
- Multiple procedures may be subject to payment reductions (MPPR)